<a href="https://colab.research.google.com/github/MargaritaKotova/Sentiment-analysis/blob/master/bert_articles_result.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime

In [62]:
!pip install bert-tensorflow

In [0]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

In [64]:
# Set the output directory for saving model file
# Optionally, set a GCP bucket location

OUTPUT_DIR = 'output_dir'#@param {type:"string"}
#@markdown Whether or not to clear/delete the directory and create a new one
DO_DELETE = True #@param {type:"boolean"}
#@markdown Set USE_BUCKET and BUCKET if you want to (optionally) store model output on GCP bucket.
USE_BUCKET = False #@param {type:"boolean"}
BUCKET = 'BUCKET_NAME' #@param {type:"string"}

if USE_BUCKET:
  OUTPUT_DIR = 'gs://{}/{}'.format(BUCKET, OUTPUT_DIR)
  from google.colab import auth
  auth.authenticate_user()

if DO_DELETE:
  try:
    tf.gfile.DeleteRecursively(OUTPUT_DIR)
  except:
    # Doesn't matter if the directory didn't exist
    pass
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

***** Model output directory: output_dir *****


### Data

First, downloading the dataset. 

In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [66]:
from google.colab import files
uploaded = files.upload()

Saving data.csv to data (1).csv


In [70]:
uploaded = files.upload()

Saving data_DataGoogleNews_labeled.csv to data_DataGoogleNews_labeled (1).csv


In [0]:
import io

In [0]:
test = pd.read_csv(io.StringIO(uploaded['data.csv'].decode('utf-8')), sep=';')

In [71]:
test.tail(6)

,Unnamed: 0,Source,Author,Title,Url,Date,Article,Polarity
172,490,BBC News,https://www.facebook.com/bbcnews,Five ways the British landscape changed in 10 ...,https://www.bbc.co.uk/news/uk-51041436,2020-01-19T01:41:49Z,"Britain is more built-on than ever, with 44.8 ...",positive
173,491,Fortune,Jeremy Kahn,Facebook wants better A.I. tools. But superint...,https://fortune.com/2020/01/20/facebook-artifi...,2020-01-20T09:30:00Z,"In early January, Facebook released software t...",positive
174,492,The Verge,James Vincent,Alphabet CEO Sundar Pichai says there is ‘no q...,https://www.theverge.com/2020/1/20/21073682/ai...,2020-01-20T10:30:05Z,Google and Alphabet CEO Sundar Pichai has call...,negative
175,493,Reuters,Foo Yun Chee and John Chalmers,Google owner calls for 'proportionate approach...,https://ca.reuters.com/article/technologyNews/...,2020-01-20T12:01:46Z,BRUSSELS (Reuters) - The EU’s proposal for a t...,negative
176,494,TechCrunch,Natasha Lomas,Google's Sundar Pichai doesn't want you to be ...,https://techcrunch.com/2020/01/20/googles-sund...,2020-01-20T14:11:15Z,"Alphabet and Google CEO, Sundar Pichai, is th...",negative
177,495,BBC News,https://www.facebook.com/bbcnews,Google boss Sundar Pichai calls for AI regulation,https://www.bbc.co.uk/news/technology-51178198,2020-01-20T15:40:00Z,The head of Google and parent company Alphabet...,negative


In [0]:
train = pd.read_csv(io.StringIO(uploaded['data_DataGoogleNews_labeled.csv'].decode('utf-8')))


In [73]:
train.tail(2)

,Source,Author,Title,Url,Date,Article,Polarity
315,Reuters,Reuters Editorial,"U.S.-Israeli startup Gong raises $65 million, ...",https://www.reuters.com/article/us-tech-gong-f...,2019-12-03T13:59:16Z,TEL AVIV (Reuters) - U.S.-Israeli startup Gong...,positive
316,Fortune,jonathanvanian2015,Arguing About Artificial Intelligence Killing ...,https://fortune.com/2019/12/03/artificial-inte...,2019-12-03T16:06:23Z,"This is the web version of Eye on A.I., Fortun...",negative


In [74]:
test.columns

Index(['Unnamed: 0', 'Source', 'Author', 'Title', 'Url', 'Date', 'Article',
       'Polarity'],
      dtype='object')

In [0]:
for i in range(len(train)):
    if train['Polarity'][i]=='negative':
            train['Polarity'][i]=0
    elif train['Polarity'][i]=='neutral':
            train['Polarity'][i]=1
    elif train['Polarity'][i]=='positive':
            train['Polarity'][i]=2       

In [76]:
for i in range(len(test)):
    if test['Polarity'][i]=='negative':
            test['Polarity'][i]=0
    elif test['Polarity'][i]=='neutral ' or test['Polarity'][i]=='neutral':
            test['Polarity'][i]=1
    elif test['Polarity'][i]=='positive ' or test['Polarity'][i]=='positive' or test['Polarity'][i]=='Positive':
            test['Polarity'][i]=2 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [77]:
train['Polarity'].value_counts()

2    146
0     98
1     73
Name: Polarity, dtype: int64

In [0]:
import numpy as np

In [0]:
for i in range(len(train)):
    if train['Polarity'][i]==np.nan:
            print(train['Polarity'][i])
   

The input data is the 'sentence' column and our label is the 'polarity' column (0, 1 for negative and positive, respecitvely)

In [0]:
train['Polarity'][168]=1

In [81]:
type(test['Polarity'][7])

int

In [82]:
test['Polarity'].unique()

array([2, 0, 1], dtype=object)

In [0]:
DATA_COLUMN = 'Article'
LABEL_COLUMN = 'Polarity'
# label_list is the list of labels, i.e. True, False or 0, 1 or 'dog', 'cat'
label_list = [0, 1, 2]

**Data** **Preprocessing**

We'll need to transform our data into a format BERT understands. This involves two steps. First, we create  InputExample's using the constructor provided in the BERT library.

text_a is the text we want to classify, which in this case, is the Request field in our Dataframe.

text_b is used if we're training a model to understand the relationship between sentences (i.e. is text_b a translation of text_a? Is text_b an answer to the question asked by text_a?). This doesn't apply to our task, so we can leave text_b blank.

label is the label for our example, i.e. True, False

In [0]:
# Use the InputExample class from BERT's run_classifier code to create examples from the data
train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

test_InputExamples = test.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)


Next, we need to preprocess our data so that it matches the data BERT was trained on. For this, we'll need to do a couple of things (but don't worry--this is also included in the Python library):

###### 1.Lowercase our text (if we're using a BERT lowercase model)
##### 2.Tokenize it (i.e. "sally says hi" -> ["sally", "says", "hi"]) 
##### 3.Break words into WordPieces (i.e. "calling" -> ["call", "##ing"]) 
##### 4.Map our words to indexes using a vocab file that BERT provides 
##### 5.Add special "CLS" and "SEP" tokens (see the readme) 
##### 6.Append "index" and "segment" tokens to each input (see the BERT paper)

In [85]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [86]:
tokenizer.tokenize("This here's an example of using the BERT tokenizer")

['this',
 'here',
 "'",
 's',
 'an',
 'example',
 'of',
 'using',
 'the',
 'bert',
 'token',
 '##izer']

In [87]:
train_InputExamples.head()

0    <bert.run_classifier.InputExample object at 0x...
1    <bert.run_classifier.InputExample object at 0x...
2    <bert.run_classifier.InputExample object at 0x...
3    <bert.run_classifier.InputExample object at 0x...
4    <bert.run_classifier.InputExample object at 0x...
dtype: object

Using our tokenizer, we'll call run_classifier.convert_examples_to_features on our InputExamples to convert them into features BERT understands.

In [88]:
# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 128
# Convert our train and test features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 317


INFO:tensorflow:Writing example 0 of 317


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: None


...

INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: 0 (id = 0)


INFO:tensorflow:label: 0 (id = 0)



**Creating a model**

create_model does just this below. First, it loads the BERT tf hub module again (this time to extract the computation graph). Next, it creates a single new layer that will be trained to adapt BERT to our sentiment task (i.e. classifying whether a movie review is positive or negative).

In [0]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  """Creates a classification model."""

  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)


Next we'll wrap our model function in a model_fn_builder function that adapts our model to work for training, evaluation, and prediction.

In [0]:
# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        f1_score = tf.contrib.metrics.f1_score(
            label_ids,
            predicted_labels)
        auc = tf.metrics.auc(
            label_ids,
            predicted_labels)
        recall = tf.metrics.recall(
            label_ids,
            predicted_labels)
        precision = tf.metrics.precision(
            label_ids,
            predicted_labels) 
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        return {
            "eval_accuracy": accuracy,
            "f1_score": f1_score,
            "auc": auc,
            "precision": precision,
            "recall": recall,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
        }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn

In [0]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 6.0
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100

In [0]:
# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

In [0]:
# Specify outpit directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [94]:
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})

INFO:tensorflow:Using config: {'_model_dir': 'output_dir', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f83915c1978>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': 'output_dir', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f83915c1978>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Next we create an input builder function that takes our training feature set (train_features) and produces a generator.

In [0]:
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

Now we train our model!

In [96]:
print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training!
INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
/tensorflow-1.15.0/python3.6/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning:

Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.



INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving checkpoints for 59 into output_dir/model.ckpt.


INFO:tensorflow:Saving checkpoints for 59 into output_dir/model.ckpt.


INFO:tensorflow:Loss for final step: 0.10846711.


INFO:tensorflow:Loss for final step: 0.10846711.


Training took time  0:42:13.575810


In [0]:
test_input_fn = run_classifier.input_fn_builder(
    features=test_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

Now let's write code to make predictions on new sentences:

In [0]:
def getPrediction(in_sentences):
  labels = [0,1,2]
  input_examples = [run_classifier.InputExample(guid="", text_a = x, text_b = None, label = 0) for x in in_sentences] # here, "" is just a dummy label
  input_features = run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
  predictions = estimator.predict(predict_input_fn)
  return [(sentence, prediction['probabilities'], labels[prediction['labels']]) for sentence, prediction in zip(in_sentences, predictions)]

In [0]:
pred_sentences = test['Article']

In [100]:
predictions = getPrediction(pred_sentences)

INFO:tensorflow:Writing example 0 of 178


INFO:tensorflow:Writing example 0 of 178


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Restoring parameters from output_dir/model.ckpt-59


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


In [101]:
predictions

[('Facebook on Friday removed what it called a global network of more than 900 accounts, pages, and groups from its platform and Instagram that allegedly used deceptive practices to push pro-Trump narratives to about 55 million users. The network used fake accounts, artificial amplification, and, notably, profile photos of fake faces generated using artificial intelligence to spread polarizing, predominantly right-wing content around the web, including on Twitter and YouTube.\nIt represents an alarming new development in the information wars, as it appears to be the first large-scale deployment of AI-generated images in a social network. In a report on the influence operation, researchers from disinformation groups Graphika and DFRLab noted that this was the first time they had seen the technology used to support an inauthentic social media campaign.\nProfile pictures for Facebook accounts “Mary Keen” and “Jacobs Guillermo,” admins on groups associated with The BL highlighted by Graphi

In [0]:
sentence=[]
polarity=[]
distribution=[]
for i in range(len(test)):
    sentence.append(predictions[i][0])
    polarity.append(predictions[i][2])
    distribution.append(predictions[i][1])


In [0]:
results=pd.DataFrame(data={'Source':test['Source'],'Author':test['Author'],'Url':test['Url'],'Date':test['Date'],'Article':test['Article'],'Title':test['Title'],'Polarity_true': test['Polarity'], 'Polarity_predict': polarity})

In [0]:
results_wide=pd.DataFrame(data={'Source':test['Source'],'Author':test['Author'],'Url':test['Url'],'Date':test['Date'],'Article':test['Article'],'Title':test['Title'],'Polarity distribution':distribution,'Polarity_true': test['Polarity'], 'Polarity_predict': polarity})

### Accuracy of the model

In [105]:
predict=0
total = len(results)
print(total)
for i in range(len(results)):
    if (results['Polarity_true'][i]==results['Polarity_predict'][i]):
        predict+=1
        
accuracy=predict/total
print(accuracy)
        

178
0.6966292134831461


In [106]:
for i in range(len(results)):
    if results['Polarity_true'][i]==0:
            results['Polarity_true'][i]='negative'
    elif results['Polarity_true'][i]==1:
            results['Polarity_true'][i]='neutral'
    elif results['Polarity_true'][i]==2:
            results['Polarity_true'][i]='positive'   
            
    if results['Polarity_predict'][i]==0:
            results['Polarity_predict'][i]='negative'
    elif results['Polarity_predict'][i]==1:
            results['Polarity_predict'][i]='neutral'
    elif results['Polarity_predict'][i]==2:
            results['Polarity_predict'][i]='positive' 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [107]:
results

,Source,Author,Url,Date,Article,Title,Polarity_true,Polarity_predict
0,Wired,Paris Martineau,https://www.wired.com/story/facebook-removes-a...,2019-12-21T01:21:05Z,Facebook on Friday removed what it called a gl...,Facebook Removes Accounts With AI-Generated Pr...,positive,negative
1,Wired,Tom Simonite,https://www.wired.com/story/ai-doctor-will-see...,2019-12-21T13:00:00Z,When MIT professor Regina Barzilay received he...,The AI Doctor Will See You Now,positive,positive
2,Wired,"Andy Greenberg, Brian Barrett",https://www.wired.com/story/facebook-two-facto...,2019-12-21T14:00:00Z,It's beginning to look a lot like the end of t...,Facebook Finally Fixes Its Two-Factor Mess,positive,positive
3,TechCrunch,Devin Coldewey,http://techcrunch.com/2019/12/21/how-to-bring-...,2019-12-21T17:47:37Z,"Artificial intelligence is a powerful tool, bu...",Luminance and Omnius are bringing AI to legacy...,positive,positive
4,The Next Web,John Gikopoulos,https://thenextweb.com/podium/2019/12/21/2020-...,2019-12-21T20:00:39Z,One of the ways you can tell that a new techno...,2020 will see AI move from buzzwords to solutions,negative,positive
...,...,...,...,...,...,...,...,...
173,Fortune,Jeremy Kahn,https://fortune.com/2020/01/20/facebook-artifi...,2020-01-20T09:30:00Z,"In early January, Facebook released software t...",Facebook wants better A.I. tools. But superint...,positive,positive
174,The Verge,James Vincent,https://www.theverge.com/2020/1/20/21073682/ai...,2020-01-20T10:30:05Z,Google and Alphabet CEO Sundar Pichai has call...,Alphabet CEO Sundar Pichai says there is ‘no q...,negative,negative
175,Reuters,Foo Yun Chee and John Chalmers,https://ca.reuters.com/article/technologyNews/...,2020-01-20T12:01:46Z,BRUSSELS (Reuters) - The EU’s proposal for a t...,Google owner calls for 'proportionate approach...,negative,negative
176,TechCrunch,Natasha Lomas,https://techcrunch.com/2020/01/20/googles-sund...,2020-01-20T14:11:15Z,"Alphabet and Google CEO, Sundar Pichai, is th...",Google's Sundar Pichai doesn't want you to be ...,negative,negative


In [108]:
results['Polarity_predict'].value_counts()

positive    123
negative     41
neutral      14
Name: Polarity_predict, dtype: int64

In [109]:
results['Polarity_true'].value_counts()

positive    101
negative     51
neutral      26
Name: Polarity_true, dtype: int64

In [0]:
from sklearn.metrics import confusion_matrix

In [111]:
confusion_matrix(results['Polarity_true'], results['Polarity_predict'])

array([[29,  5, 17],
       [ 5,  5, 16],
       [ 7,  4, 90]])

In [112]:
results_wide['Polarity distribution']

0        [-0.5121273, -2.8696733, -1.0669303]
1       [-4.0325685, -4.2178593, -0.03299731]
2        [-1.710292, -1.7299899, -0.44332144]
3       [-4.8914614, -4.3072643, -0.02120397]
4        [-1.0109742, -1.5238645, -0.8716344]
                        ...                  
173    [-4.9799657, -4.2542167, -0.021303868]
174       [-0.0820126, -3.5243335, -3.010478]
175     [-0.10035335, -2.7777216, -3.4021297]
176    [-0.091060586, -2.6519723, -4.1028414]
177     [-0.06503656, -3.1993215, -3.8086987]
Name: Polarity distribution, Length: 178, dtype: object

In [0]:
from sklearn.metrics import log_loss

In [0]:
from scipy.special import softmax

In [0]:
distribution_01 =softmax(distribution)

In [116]:
distribution_01

array([[3.36640095e-03, 3.18637496e-04, 1.93293835e-03],
       [9.95996743e-05, 8.27536060e-05, 5.43562463e-03],
       [1.01580331e-03, 9.95989656e-04, 3.60618485e-03],
       [4.21934710e-05, 7.56761146e-05, 5.50010754e-03],
       [2.04418204e-03, 1.22397882e-03, 2.34981626e-03],
       [8.24053568e-05, 3.76082462e-04, 5.15948934e-03],
       [4.11400724e-05, 6.79454388e-05, 5.50889224e-03],
       [7.26787257e-05, 3.16579943e-04, 5.22871781e-03],
       [2.11284481e-04, 1.57796880e-04, 5.24889491e-03],
       [8.96581332e-04, 2.50287791e-04, 4.47110785e-03],
       [3.83548485e-03, 3.31659685e-04, 1.45083235e-03],
       [6.17003127e-04, 3.89711873e-04, 4.61126119e-03],
       [1.02959608e-03, 6.72341790e-04, 3.91603913e-03],
       [3.88703193e-04, 3.90542089e-04, 4.83873067e-03],
       [1.50289081e-04, 7.60836352e-04, 4.70685167e-03],
       [2.52623722e-04, 1.16132847e-04, 5.24922041e-03],
       [8.05347590e-05, 1.90780978e-04, 5.34666004e-03],
       [1.35272494e-04, 6.69684

In [120]:
!pip install chart_studio

     |████████████████████████████████| 81kB 2.3MB/s 


In [121]:
from plotly.offline import iplot, init_notebook_mode
# standard plotly imports
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
# using plotly + cu|fflinks in offline mode
import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)

import matplotlib.pyplot as plt

In [122]:
results['Polarity_true'].iplot(
    kind='hist',
    histnorm='percent',
    barmode='overlay',
    xTitle='Positivity',
    yTitle='Number of articles')

In [123]:
results['Polarity_predict'].iplot(
    kind='hist',
    histnorm='percent',
    barmode='overlay',
    xTitle='Positivity',
    yTitle='Number of articles')

In [0]:
results.to_csv('results_bert.csv')

In [126]:
results_wide.head()

,Source,Author,Url,Date,Article,Title,Polarity distribution,Polarity_true,Polarity_predict
0,Wired,Paris Martineau,https://www.wired.com/story/facebook-removes-a...,2019-12-21T01:21:05Z,Facebook on Friday removed what it called a gl...,Facebook Removes Accounts With AI-Generated Pr...,"[-0.5121273, -2.8696733, -1.0669303]",2,0
1,Wired,Tom Simonite,https://www.wired.com/story/ai-doctor-will-see...,2019-12-21T13:00:00Z,When MIT professor Regina Barzilay received he...,The AI Doctor Will See You Now,"[-4.0325685, -4.2178593, -0.03299731]",2,2
2,Wired,"Andy Greenberg, Brian Barrett",https://www.wired.com/story/facebook-two-facto...,2019-12-21T14:00:00Z,It's beginning to look a lot like the end of t...,Facebook Finally Fixes Its Two-Factor Mess,"[-1.710292, -1.7299899, -0.44332144]",2,2
3,TechCrunch,Devin Coldewey,http://techcrunch.com/2019/12/21/how-to-bring-...,2019-12-21T17:47:37Z,"Artificial intelligence is a powerful tool, bu...",Luminance and Omnius are bringing AI to legacy...,"[-4.8914614, -4.3072643, -0.02120397]",2,2
4,The Next Web,John Gikopoulos,https://thenextweb.com/podium/2019/12/21/2020-...,2019-12-21T20:00:39Z,One of the ways you can tell that a new techno...,2020 will see AI move from buzzwords to solutions,"[-1.0109742, -1.5238645, -0.8716344]",0,2


In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import files
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
results_wide.to_csv("results_bert_wide.csv")


In [0]:
distribution1=[]
distribution2=[]
distribution3=[]
for i in range(len(distribution)):
    distribution1.append(distribution_01[i][0])
    distribution2.append(distribution_01[i][1])
    distribution3.append(distribution_01[i][2])

In [0]:
results_wide_1=pd.DataFrame(data={'Source':test['Source'],'Author':test['Author'],'Url':test['Url'],'Date':test['Date'],'Article':test['Article'],'Title':test['Title'],'Polarity distribution1':distribution1,'Polarity distribution2':distribution2,'Polarity distribution3':distribution3, 'Polarity_true': test['Polarity'], 'Polarity_predict': polarity})

In [132]:
#result_wide_1['Max probability']=max(results_wide_1['Polarity distribution1'], results_wide_1['Polarity distribution2'],  results_wide_1['Polarity distribution3'])
result_wide_1['Max probability'] = results_wide_1[['Polarity distribution1','Polarity distribution2','Polarity distribution3']].apply(np.max,axis=1)

NameError: ignored

In [133]:
results_wide_1['Polarity distribution1']

0      0.003366
1      0.000100
2      0.001016
3      0.000042
4      0.002044
         ...   
173    0.000039
174    0.005176
175    0.005082
176    0.005129
177    0.005264
Name: Polarity distribution1, Length: 178, dtype: float64

In [134]:
results_wide_1.head()

,Source,Author,Url,Date,Article,Title,Polarity distribution1,Polarity distribution2,Polarity distribution3,Polarity_true,Polarity_predict
0,Wired,Paris Martineau,https://www.wired.com/story/facebook-removes-a...,2019-12-21T01:21:05Z,Facebook on Friday removed what it called a gl...,Facebook Removes Accounts With AI-Generated Pr...,0.003366,0.000319,0.001933,2,0
1,Wired,Tom Simonite,https://www.wired.com/story/ai-doctor-will-see...,2019-12-21T13:00:00Z,When MIT professor Regina Barzilay received he...,The AI Doctor Will See You Now,0.000100,0.000083,0.005436,2,2
2,Wired,"Andy Greenberg, Brian Barrett",https://www.wired.com/story/facebook-two-facto...,2019-12-21T14:00:00Z,It's beginning to look a lot like the end of t...,Facebook Finally Fixes Its Two-Factor Mess,0.001016,0.000996,0.003606,2,2
3,TechCrunch,Devin Coldewey,http://techcrunch.com/2019/12/21/how-to-bring-...,2019-12-21T17:47:37Z,"Artificial intelligence is a powerful tool, bu...",Luminance and Omnius are bringing AI to legacy...,0.000042,0.000076,0.005500,2,2
4,The Next Web,John Gikopoulos,https://thenextweb.com/podium/2019/12/21/2020-...,2019-12-21T20:00:39Z,One of the ways you can tell that a new techno...,2020 will see AI move from buzzwords to solutions,0.002044,0.001224,0.002350,0,2


In [0]:
results_wide_1.to_csv('results_bert_wide1.csv') 
files.download('results_bert_wide1.csv')